# L6d: Multiplicative Weight Update Algorithm for Linear Programs
In this lab, we'll use a variant of the [Multiplicative Weight Update Algorithm](https://en.wikipedia.org/wiki/Multiplicative_weight_update_method) to (approximately) solve a [linear programming problem](https://en.wikipedia.org/wiki/Multiplicative_weight_update_method#Solving_linear_programs_approximately[14]). 
Let $\Delta_{m} = \{\mathbf{x} \in \mathbb{R}_{\geq{0}}^{m} \mid \sum_{i=1}^{m}x_{i} = \tau\}$ be a set of $m$-dimensional vectors with non-negative entries that sum to $\tau$. 
$$
\begin{align*}
\text{Find} &\quad \mathbf{x} \in \Delta_{m} \\
\text{subject to} &\quad \mathbf{A}\mathbf{x} \leq \mathbf{b}
\end{align*}
$$
This formulation may seem restrictive, but we can convert _most_ linear programs into this form. We will use the Multiplicative Weight Update algorithm to solve the following problem. There is a famous (Cornell) Multiplicative Weight Update algorithm to solve this problem:
* [Plotkin, Serge A., et al. “Fast Approximation Algorithms for Fractional Packing and Covering Problems.” Mathematics of Operations Research, vol. 20, no. 2, 1995, pp. 257–301](https://www.jstor.org/stable/3690406?socuuid=57de56c3-135d-4376-9af5-be0257a4c2d8)

but we will use our own implementation [inspired by a lecture by Prof. Saranurak at the University of Michigan](https://www.youtube.com/watch?v=5u8wYZjsHuc&t=3190s).


### Algorithm
__Initialize__: We have $m$ experts (one for each unknown $x_{i}$) and $T$ rounds. Each expert $i$ has a weight $w_{i}^{(t)}$ at round $t$. The weights are initialized to $w_{i}^{(1)}=1$ for all experts. We specify a learning rate $\eta\in{(0,1)}$. The constraint matrix $\mathbf{A}\in\left[-\rho,\rho\right]^{n\times{m}}$ and $\mathbf{b}\in\mathbb{R}^{m}$.

For each round $t=1,2,\dots,T$ where the number of rounds is $T = \ln({m})/\epsilon^{2}$:
1. The aggregator (us) computes a candidate solution $x_{i}^{(t)} = \tau\left(w_{i}^{(t)}/\Phi^{(t)}\right)$ from the weight of each expert $i$, where $\Phi^{(t)} = \sum_{i=1}^{m}w_{i}^{(t)}$.
2. The aggregator (us) computes the associated constraint violation (loss) $\mathbf{r} = \mathbf{A}\mathbf{x}^{(t)} - \mathbf{b} -2\epsilon\mathbf{I}$. If all $\{r_{i} \leq 0 \mid i = 1,2,\dots,n\}$ we have a feasible solution. We return $\mathbf{x}^{(t)}$ and `true.` Otherwise, if _any_ elements $\{r_{i}>0 \mid i = 1,2,\dots,n\}$ we update the weights of the experts.
3. For each violated constraint $k\in\left[n\right]$, i.e., $\{k\}$ are the indexed with $r_{i}>0$, we update the weights of _all experts_ using the update rule:
$$
\begin{align*}
w_{j}^{(t+1)} = w_{j}^{(t)}\cdot\left(1-\eta\cdot{a_{k,j}}\right) \quad j = 1,2,\dots,m
\end{align*}
$$
4. Go to step 1. We iterate until we find a solution that satisfies all constraints, or we run out of rounds. If we run out of rounds without finding a solution, we return `false,` which says our problem formulation is infeasible.

If the problem is feasible, this approach will converge to a solution $\mathcal{O}((\dim\mathbf{A}_{\neq{0}})\cdot\ln(m)\cdot(\rho\tau/\epsilon)^{2})$

### Tasks
Before we start, divide into teams and familiarize yourself with the lab. Then, execute the `Run All Cells` command to check if you (or your neighbor) have any code or setup issues. Code issues, then raise your hands - and let's get those fixed!
* __Task 1: Setup, Data, Prerequisites (10 min)__: Let's take 10 minutes to set up the problem and, in particular, look at the graph we will explore.
* __Task 2: Set up the problem model and play the game (20 min)__: In this task, we'll set up and solve the linear programming problem. First, we'll build an [instance of the `MyConstraintCheckingGameModel` type](src/Types.jl) containing information about the problem. Then, we'll solve the problem and think about the results.
* __Task 3: Check and analyze the results (10 min)__: In this task, we analyze the results produced by our approach. First, we check the bounds for constraint violations and then examine the solution to see if it makes sense (and what the algorithm returns). We'll start with the bounds and then consider the properties of the flow vector.

## Task 1: Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 
* The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [1]:
include("Include.jl");

  Activating project at `C:\Users\danie\CHEME5820\CHEME-5820-Labs-Spring-2025\labs\week-6\L6d`
    Updating `C:\Users\danie\CHEME5820\CHEME-5820-Labs-Spring-2025\labs\week-6\L6d\Project.toml`
  [336ed68f] + CSV v0.10.15
  [5ae59095] + Colors v0.13.0
  [a93c6f00] + DataFrames v1.7.0
  [31c24e10] + Distributions v0.25.117
  [5789e2e9] + FileIO v1.16.6
  [ec8451be] + KernelFunctions v0.10.64
  [91a5bcdd] + Plots v1.40.9
  [08abe8d2] + PrettyTables v2.4.0
  [10745b16] + Statistics v1.11.1
  [f3b207a7] + StatsPlots v0.15.7
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
    Updating `C:\Users\danie\CHEME5820\CHEME-5820-Labs-Spring-2025\labs\week-6\L6d\Manifest.toml`
  [621f4979] + AbstractFFTs v1.5.0
  [79e6a3ab] + Adapt v4.2.0
  [66dad0bd] + AliasTables v1.1.3
  [7d9fca2a] + Arpack v0.5.4
  [13072b0f] + AxisAlgorithms v1.1.0
  [d1d4a3ce] + BitFlags v0.1.9
  [336ed68f] + CSV v0.10.15
  [d360d2e6] + ChainRulesCore v1.25.1
  [aaaa29a8] + Clustering v0.15.8
  [944b1d66] + CodecZlib v0.7.8
  [35d6a980] 

  [975044d2] + Xorg_xcb_util_keysyms_jll v0.4.0+1
  [0d47668e] + Xorg_xcb_util_renderutil_jll v0.3.9+1
  [c22f9ab0] + Xorg_xcb_util_wm_jll v0.4.1+1
  [35661453] + Xorg_xkbcomp_jll v1.4.6+1
  [33bec58e] + Xorg_xkeyboard_config_jll v2.39.0+0
  [c5fb5394] + Xorg_xtrans_jll v1.5.1+0
  [3161d3a3] + Zstd_jll v1.5.7+0
  [35ca27e7] + eudev_jll v3.2.9+0
  [214eeab7] + fzf_jll v0.56.3+0
  [1a1c6b14] + gperf_jll v3.1.1+1
  [a4ae2306] + libaom_jll v3.11.0+0
  [0ac62f75] + libass_jll v0.15.2+0
  [1183f4f0] + libdecor_jll v0.2.2+0
  [2db6ffa8] + libevdev_jll v1.11.0+0
  [f638f0a6] + libfdk_aac_jll v2.0.3+0
  [36db933b] + libinput_jll v1.18.0+0
  [b53b4c65] + libpng_jll v1.6.46+0
  [f27f6e37] + libvorbis_jll v1.3.7+2
  [009596ad] + mtdev_jll v1.1.6+0
  [1317d2d5] + oneTBB_jll v2022.0.0+0
⌅ [1270edf5] + x264_jll v2021.5.5+0
⌅ [dfaa095f] + x265_jll v3.5.0+0
  [d8fb68d0] + xkbcommon_jll v1.4.1+2
  [0dad84c5] + ArgTools v1.1.2
  [56f22d72] + Artifacts v1.11.0
  [2a0f44e3] + Base64 v1.11.0
  [ade2ca70] + 

### Graph model 

We'll represent the network we will explore as the graph $\mathcal{G}=\left(\mathcal{V},\mathcal{E}\right)$ which we model as [an instance of the `MySimpleDirectedGraphModel` type](src/Types.jl). We create our graph model from an [Edge list representation](https://en.wikipedia.org/wiki/Edge_list). 
* In the [Edge list representation](https://en.wikipedia.org/wiki/Edge_list), only the edge information is stored (typically) in a comma-separated value (CSV) file in which each record holds an edge in the graph, and the fields contain `source, target, ....` data for each edge.
* We've used [the `readedgesfile(...)` function in `src/Files.jl`](src/Files.jl) to build a list of edges in our graph, where each edge is an instance of [the `MyGraphEdgeModel` type](src/Types.jl) which holds edge information.
* We can then pass the edge list to [a `build(...)` method](src/Factory.jl) for our graph model, the populated `graphmodel::MySimpleDirectedGraphModel` is returned.

In [2]:
graphmodel = let

    # load up the balanced example -
    balanced_edgefile = joinpath(_PATH_TO_DATA, "Network.edgelist");
    balanced_graphmodel = readedgesfile(balanced_edgefile) |> edges -> build(MySimpleDirectedGraphModel, edges);

    balanced_graphmodel
end;

What's in the `graphmodel::MySimpleDirectedGraphModel` instance?

In [3]:
graphmodel

MySimpleDirectedGraphModel(Dict{Int64, MyGraphNodeModel}(5 => MyGraphNodeModel(5), 4 => MyGraphNodeModel(4), 2 => MyGraphNodeModel(2), 3 => MyGraphNodeModel(3), 1 => MyGraphNodeModel(1)), Dict((4, 5) => (1.0, 0.0, 1.0), (1, 2) => (1.0, 0.0, 1.0), (2, 5) => (1.0, 0.0, 1.0), (1, 3) => (1.0, 0.0, 1.0), (1, 4) => (1.0, 0.0, 1.0), (3, 5) => (1.0, 0.0, 1.0)), Dict(5 => (3, 5), 4 => (2, 5), 6 => (4, 5), 2 => (1, 3), 3 => (1, 4), 1 => (1, 2)), Dict{Int64, Set{Int64}}(5 => Set(), 4 => Set([5]), 2 => Set([5]), 3 => Set([5]), 1 => Set([4, 2, 3])), [-1.0 -1.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 -1.0; 0.0 0.0 … 1.0 1.0])

### Constants and other parameters
Before solving the linear program, we need to specify a number of items. Let's begin with how much error we are willing to live with, and then we'll move on to the components of the constraints $\mathbf{A}\mathbf{x}\leq\mathbf{b}$.

Let's start with the tolerance parameter `ϵ::Float64`:

In [4]:
ϵ = 0.01; # we willing to live with a 2ϵ violation of *every* constraint

Next, let's specify the matrix $\mathbf{A}\in\mathbb{R}^{n\times{m}}$ and the right-hand side vector $\mathbf{b}\in\mathbb{R}^{n}$.

The constraint matrix $\mathbf{A}$ is [the incidence matrix of our graph $\mathcal{G}$]( https://en.wikipedia.org/wiki/Incidence_matrix). The incidence matrix of a directed graph is a $n\times{m}$ matrix $\mathbf{A}$ where n and m are the number of vertices and edges, respectively, such that:
$$
\begin{equation}
a_{ij} = \begin{cases}
-1 & \text{if edge}\,e_{j}\,\text{leaves vertex}\,v_{i}\\
1 & \text{if edge}\,e_{j}\,\text{enters vertex}\,v_{i}\\
0 & \text{otherwise}
\end{cases}
\end{equation}
$$
The incidence matrix was already constructed for us when we built the `graphmodel::MySimpleDirectedGraphModel` instance; it is stored in the `A` field of the graph model.

The right-hand side vector $\mathbf{b}\in\mathbb{R}^{n}$ holds the right side of the inequalities. We'll set a default of `0` and then adjust the entries for different problems (or when we are playing with the hyperparameters of the approach).

In [5]:
A, b = let

    A = graphmodel.A # let's get A
    n = size(A,1); # number of nodes (rows)
    m = size(A,2); # number of edges (cols)
    
    # right-hand side -
    b = zeros(n); # number of nodes
    b[1] = 1.0;
    b[2] = 0.1;
    b[end] = 1.0

    A,b
end;

## Task 2: Set up the problem model and play the game.
In this task, we'll set up and solve the linear programming problem. First, let's build an [instance of the `MyConstraintCheckingGameModel` type](src/Types.jl) containing information about the problem. Then, we'll solve the problem and think about the results.

The [`MyConstraintCheckingGameModel` type](src/Types.jl) has several fields related to the problem. To construct this type, we pass the type of thing we want to construct and information about the problem to [a `build(...)` method](src/Factory.jl). This returns the populated model in the `model::MyConstraintCheckingGameModel` variable.

In [6]:
model = let

    # get data from the graph model -
    A = graphmodel.A; # constraint matrix
    n = size(A,1); # number of nodes (constraints)
    m = size(A,2); # number of edges (variables)
    T = log(m)/(ϵ^2) |> x-> round(Int,x) # number of time steps

    # build the problem model -
    model = build(MyConstraintCheckingGameModel, (
        η = 0.05, # learning rate
        T = T, # max number of time steps
        A = A, # constraint matrix
        b = b, # right-hand side vector
        ρ = 1.0, # max element in A (scale: [-ρ,ρ])
        τ = 1.0, # sum of x
    ));

    # return -
    model;
end;

### Play the game 

To play the game, i.e., to solve the linear programming problem, we pass the `model::MyConstraintCheckingGameModel` instance and our tolerance value `ϵ::Float64` to [the `play(...)` method](src/Online.jl). This method returns several pieces of data that are interesting:
* The $\mathbf{x}\in\mathbb{R}^{m}$ vector is the best solution found for the problem; we store this in the `x::Array{Float64,1}` variable.
* The `flag::Bool = {true | false}` variable tells us where or not the aggregator (us) found a solution (or ran out of iterations).
* Finally, the `w::Array{Float64,2}` variable holds the weight matrix, where each row is an iteration, and the columns correspond to the weight of expert $i$ (one expert per variable).

However, before we move on to the next task, we check convergence [using the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert). If the aggregator (us) fails to find a solution, [an `AssertionError` is thrown](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError).

In [7]:
x,flag,w = play(model, ϵ = ϵ);
@assert flag == true

## Task 3: Check and analyze the results
In this task, we analyze the results produced by our approach. First, we check the bounds for constraint violations and then look at the solution to see if it makes sense (and what the algorithm returned). Let's start with the bounds.

### Bounds table 
`Unhide` the code block below to see the residual that the algorithm produced for each constraint $r_{i} = \sum_{j}a_{ij}x_{j} - b_{i}$, the specified residual $b_{i}$ and whether the constraint was violated.
* __Summary__: If the constraint was violated, then `flag = false,` so we should have seen [an `AssertionError`](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError). Otherwise $r_{i}\leq{b}_{i}\,\forall{i}$.

In [8]:
let
    df = DataFrame();
    r₁ = A*x;

    for i ∈ eachindex(r₁)
        value = r₁[i];
        row_df = (
            node = i,
            rᵢ = value,
            bᵢ = b[i],
            violationᵢ = (value - b[i]) > 2*ϵ ? true : false
        );
        push!(df, row_df);
    end

    pretty_table(df, tf = tf_simple)
end

======== ========= ========= =============
   node        rᵢ        bᵢ   violationᵢ 
  Int64   Float64   Float64         Bool 
======== ========= ========= =============
      1      -0.5       1.0        false
      2       0.0       0.1        false
      3       0.0       0.0        false
      4       0.0       0.0        false
      5       0.5       1.0        false
======== ========= ========= =============


### Edge table
`Unhide` the code block below to see the value for the flow carried on each edge of our graph $\mathcal{G}$.
* __Summary__: The sum of the flows should be: $\sum_{j}x_{j} = \tau$. We check the summation condition below [using the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert). If the summation condition is violated [an `AssertionError` is thrown](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError).

In [9]:
let

    ei = graphmodel.edgesinverse;
    df = DataFrame()

    for i ∈ eachindex(x)
        flow = x[i];
        edgetuple = ei[i];

        row_df = (
            edge = i,
            start = edgetuple[1],
            stop = edgetuple[2],
            flow = flow
        );
        push!(df, row_df);
    end

    pretty_table(df, tf=tf_simple)
end

======== ======= ======= ===========
   edge   start    stop       flow 
  Int64   Int64   Int64    Float64 
======== ======= ======= ===========
      1       1       2   0.166667
      2       1       3   0.166667
      3       1       4   0.166667
      4       2       5   0.166667
      5       3       5   0.166667
      6       4       5   0.166667
======== ======= ======= ===========


In [10]:
@assert sum(x) ≈ model.τ